In [ ]:
import plotly
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import collections

plotly.tools.set_credentials_file(username='Fernet', api_key='zC7ogVfVT5UPcpu2VKse')
plotly.offline.init_notebook_mode(connected=True)

#---------------- Settings ----------------

image_ext = "svg" #Extension para bajar los graficos
poda = 5 #Cuantos datos se dejan a cada lado al podar
image_h = 1000
image_w = 1400

#---------------- Funciones ----------------

#Defino tupla con nombre, para devolver varios argumentos
Return_data = collections.namedtuple("Return_data", ["Tam", "Prom"])


#Toma el archivo a leer y si se quiere ordenar por tam o por tiempo promedio. Por default, se ordenan por num_tira


def procesar_csv(data_path):
    df = pd.read_csv(data_path, sep = ";")
    cant_exp = df.shape[0] #Shape = (filas, columnas)
    proms = []
    #Ordeno las instancias por tiempo
    df.sort_values("Tiempo", inplace = "True")
    #Descarto los valores en los extremos
    df = df.iloc[poda : cant_exp - poda, :]

    promedio = df["Tiempo"].mean(axis = 1) #Promedio podado de las instancias
    v = df["V"][0]    
    
    
    ret_data = Return_data(v, promedio)
    
    return ret_data

def procesar_carpeta(folder_path):
    tam []
    proms []
    for root, dirs, files in os.walk(folder_path):
        with open(file) as file:
            if file.endswith(".csv"):
                datos = procesar_csv(file)
                tam.append(datos.Tam)
                proms.append(datos.Prom)
    
    dict = {"tam": tam, "proms": proms}
    df = pd.DataFrame(data = dict)
    return df
                

#Toma un vector de tuplas (x, y, nombre) y devuelve una figura con cada tupla de datos
def crear_fig(traces, xtitle, ytitle, fig_title):
    data = []
    for trace in traces:
        rand_o_color = "line" #Si es tira azul o roja, no quiero markers
        if(trace[2].find("az") != -1 and trace[2].find("ro") != -1):
            #Si es tira random, si
            rand_o_color = "line+markers"
        new_trace = go.Scatter(
            x = trace[0],
            y = trace[1],
            name = trace[2],
            mode = rand_o_color
                
        )
        data.append(new_trace)
    
    layout = go.Layout(
        xaxis=dict(
            tickmode = "auto",
            #nticks = cant_exp,
            ticks = "inside",
            #tick0 = 0,
            #ticklen = 8,
            #dtick = 3,
            title = xtitle,
            tickangle = 0
        ),
        yaxis=dict(
            tick0 = 0,
            zeroline = "False",
            ticks='inside',
            ticklen=8,
            tickwidth=3,
            tickcolor='#000',
            title = ytitle
        ),
        legend=dict(
            x=10,
            y=1,
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=12,
                color='#000'
            ),
            bgcolor='#E2E2E2',
            bordercolor='#FFFFFF',
            borderwidth=2
        ),
        title = fig_title
    )
    
    return go.Figure(data=data, layout=layout)

#---------------- Script ----------------

#Procesamiento de datos

data_ejer1 = procesar_carpeta("Datos/Ej1/nVariable")


#Traces
back_1_rand_trace = (back_1_rand_tam, back_1_rand_tiempos, "back_1_rand")


#Graficos


nombre = "Nombre del grafico"

traces = []

fig = crear_fig(traces, "Tam. tira", "Tiempo promedio (ns)", nombre)

plotly.offline.iplot(fig, image = image_ext, filename = "Nombre del archivo que descarga")

